# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [1]:
!pip install pandarallel


  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=96692c4a05e9696c3757c85467511a314259fa677028404b741493bd48d82a94
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [2]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet
/kaggle/input/later-data/Full_merged.pkl
/kaggle/input/later-data/full_clustered .pkl
/kaggle/input/later-data/cluster_model.joblib
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [ ]:
df=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [ ]:
events['step'].isnull().sum()

In [ ]:
event_df=events.dropna()

In [ ]:
event_df

In [ ]:
df['window']=np.nan

In [ ]:
ind=df.index[df['event'].isna()==False]
print(ind)

In [ ]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-360
    b=i+360
    df['window'].loc[a:i]=int(c)
    c=c+1
    df['window'].loc[i:b]=int(c)
    c=c+1
print('Windows Created: ', c)    



In [ ]:
df=df[df['window'].isna()==False]

In [ ]:
df['window'].isnull().sum()

In [15]:
#df.to_pickle('ts_window_df60.pkl')
df=pd.read_pickle('ts_window_df60.pkl')

In [16]:
df.describe()

,step,anglez,enmo,sleep,cluster,window
count,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06
mean,2.142430e+05,-9.707589e+00,2.275398e-02,4.998330e-01,1.501805e+00,4.791930e+03
std,1.412759e+05,4.350354e+01,5.409548e-02,5.000000e-01,1.259533e+00,2.766938e+03
min,0.000000e+00,-9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.530800e+04,-4.534660e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.396000e+03
50%,2.004120e+05,-1.312360e+01,4.500000e-03,0.000000e+00,1.000000e+00,4.792000e+03
75%,3.173210e+05,2.323820e+01,2.480000e-02,1.000000e+00,3.000000e+00,7.188000e+03
max,7.397520e+05,8.998630e+01,5.507400e+00,1.000000e+00,3.000000e+00,9.584000e+03


In [ ]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

# Normalizing cluster and sleep

In [17]:
df_n=df
df_n['cluster']=df['cluster']/6
df_n['sleep']=df['sleep']/5
df_n.describe()

,step,anglez,enmo,sleep,cluster,window
count,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06,6.910640e+06
mean,2.142430e+05,-9.707589e+00,2.275398e-02,9.996660e-02,2.503009e-01,4.791930e+03
std,1.412759e+05,4.350354e+01,5.409548e-02,1.000000e-01,2.099222e-01,2.766938e+03
min,0.000000e+00,-9.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9.530800e+04,-4.534660e+01,0.000000e+00,0.000000e+00,0.000000e+00,2.396000e+03
50%,2.004120e+05,-1.312360e+01,4.500000e-03,0.000000e+00,1.666667e-01,4.792000e+03
75%,3.173210e+05,2.323820e+01,2.480000e-02,2.000000e-01,5.000000e-01,7.188000e+03
max,7.397520e+05,8.998630e+01,5.507400e+00,2.000000e-01,5.000000e-01,9.584000e+03


## Analyzing the window

In [18]:
for i in range(10):
        fig=px.line(y=df_n['enmo'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['cluster'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['sleep'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])

        fig.show()

In [20]:
df.resample('60min', label='right').first()


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [ ]:
fig=px.line(y=df['enmo'][df['window']==4],x=df['timestamp'][df['window']==4])
fig.add_scatter(y=df['cluster'][df['window']==4],x=df['timestamp'][df['window']==4])
fig.show()

In [ ]:
fig=px.line(y=df['enmo'][df['window']==2],x=df['timestamp'][df['window']==2])
fig.add_scatter(y=df['cluster'][df['window']==2],x=df['timestamp'][df['window']==2])
fig.show()

In [ ]:
fig=px.line(y=df['enmo'][df['window']==1],x=df['timestamp'][df['window']==1])
fig.add_scatter(y=df['cluster'][df['window']==1],x=df['timestamp'][df['window']==1])
fig.show()